# NI-MPI - Domácí úkol ZS 2021/2022
#### Author:   Oleksandr Korotetskyi
#### Date:   27.11.2021


## Results
### Mandatory variants

| Variant | Iterations | &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Variant&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | Comment |
|:--------:|:-------------:|:--------------:|:--------------------------------:|
| Jacobi a | 15            | $\gamma = 5$   |                                  |
| Jacobi b | 987           | $\gamma = 2$   |                                  |
| Jacobi c | -             | $\gamma = 0.5$ | Iteration method does not converge |
| Gauss-Seidel a | 10 | $\gamma = 5$ |  |
| Gauss-Seidel b | 495 | $\gamma = 2$ |  |
| Gauss-Seidel c | - | $\gamma = 0.5$ | Iteration method does not converge |

### Volitelné varianty s velikostí matice
| Variant | Iterations | &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Variant&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | Comment |
|:-:|:-:|:-:|:-:|
| Jacobi 2.001 | 949 | $\gamma = 2.001$ |  |
| Jacobi 2.004 | 850 | $\gamma = 2.004$ |  |
| Jacobi 4 | 20 | $\gamma = 4$   |  |
| Jacobi 16 | 7 | $\gamma = 16$   |  |
| Jacobi 1024 | 3 | $\gamma = 1024$ |  |
| Gauss-Seidel $2.001$ | 476 | $\gamma = 2.001$ |  |
| Gauss-Seidel $2.004$ | 427 | $\gamma = 2.004$ |  |
| Gauss-Seidel 4 | 13 | $\gamma = 4$ |  |
| Gauss-Seidel 16 | 6 | $\gamma = 16$ |  |
| Gauss-Seidel 1024 | 2 | $\gamma = 1024$ |  |
| Gauss-Seidel $\omega = 0.5$ | 1488 | $\gamma = 2$, $\omega = 0.5$ |  |
| Gauss-Seidel $\omega = 1.1$ | 405 | $\gamma = 2$, $\omega = 1.1$ |  |
| Gauss-Seidel $\omega = 1.5$ | 161 | $\gamma = 2$, $\omega = 1.5$ |  |
| Gauss-Seidel $\omega = 2$ | - | $\gamma = 2$, $\omega = 2$ | Iteration method does not converge |

## Code

The developed solution represents a toolchain for computing the number of iterations required to solve the system of equations using Jacobi and Gauss-Seigel methods. The toolchain itself consists of the following executables:
1. `EquationSolver.py` - the very python3 script used for task-defined computation
2. `run.sh` - bash script which is the core of a very toolchain. Is used for performing the calculations in the specified range of a Gamma variable.
3. `plotter.py` - python3 script to produce the vizualization of iterations number dependence upon the Gamma value. 

#### _How to run the toolchain?_

The toolchain requires some additional command line arguments to run:
1. _start_ - the initial value of gamma
2. _step_  - represents the increase of a gamma value per each iteration
3. _stop_  - the final value of gamma to computate
4. _omega_ - omega parameter used in Gauss-Seidel method (set to 1 by default)

Example: `bash ./run.sh 2 0.01 2.2 1` (increase the gamma value from 2 by 0.01 per each iteration, until gamma reaches the value of 2.2, keeping _omega_ set to 1)

#### _How it works?_
The toolchain runs `EquationSolver.py` for each value of gamma in the specified range for both of Jacobi and Gauss-Seidel methods (omega is fixed), storing the results of each computation in temporary files. After all gammas in specified range were calculated, the script runs the visualization utility `plotter.py` which plots the number of iterations for each gamma value for both of methods.

#### _How to run EquationSolver?_
The very `EquationSolver.py` itself can be run without toolchain. Requires the following arguments:
1. _gamma_ - gamma value to calculate for
2. _method_ - `True` for Jacobi, `False` for Gauss-Seigel methods
3. _omega_ - the omega value used in Gauss_Seigel method

Example: `python3 EquationSolver.py 2.67 False 1`

The demonstration of _EquationSolver_ is provided below.


### EquationSolver


In [87]:
import sys
import numpy as np
from pprint import pprint

class EquationSolver:

    # Initiates the instance for specific gamma value
    def __init__(self, gamma):

        sys.setrecursionlimit(10000)
        
        # matrix A
        self.a = np.zeros((20, 20), dtype=np.float64)
        np.fill_diagonal(self.a, gamma)
        np.fill_diagonal(self.a[:, 1:], -1)
        np.fill_diagonal(self.a[1:], -1)

        # vector b
        self.b = np.full((20, 1), gamma-2, dtype=np.float64)
        self.b[0], self.b[-1] = gamma - 1, gamma - 1

        # matrix D
        self.d = np.diag(np.diag(self.a))

        # matrix L
        self.l = np.tril(self.a, -1)

        # matrix U
        self.u = self.a - self.d - self.l

        # set default precision
        self.precision = 10**-6


    # Sets the calculation method
    def set_method(self, method, omega = 1):

        self.q = self.d if method else (self.d / omega) + self.l
        self.q_inv = np.linalg.inv(self.q)
        self.convergent = None
        self.result = None
        self.iterations = None

        return self
    

    # Checks if iterative method is convergent and computes the result
    def compute(self):

        if not self.is_convergent():
            self.result = []
            self.iterations = 0
        if self.iterations is None:
            self.result, self.iterations = self.calculate()

        return self.result, self.iterations


    # Calculates convergence of iterative method
    def is_convergent(self):

        if self.convergent is None:                                                         
            self.convergent = max(abs(np.linalg.eigvals(np.identity(20, dtype=np.float64) - (self.q_inv @ self.a)))) < 1

        return self.convergent


    # Calculate with the iterative method
    def calculate(self, xn=None, idx=0):

        if xn is None:
            xn = np.zeros((20, 1), np.float64)
        if self.is_precision_sufficient((self.a @ xn) - self.b):
            return xn.transpose(), idx
        
        return self.calculate(self.calc_it(xn), idx + 1)

    
    # Checks whether the precision of resuslt is satisfactory
    def is_precision_sufficient(self, result):

        return self.precision > (np.linalg.norm(result) / np.linalg.norm(self.b))


    # Calculates single iteration, derived into single function to reduce recursion
    def calc_it(self, xn):

        return (self.q_inv @ (((self.q - self.a) @ xn) + self.b))

## Demonstration of functionality

In [90]:
# Create a solver class and pass a gamma value as a parameter.
es = EquationSolver(2.3)

# Set the method which is used for a computation
# True: Jacobi method; does not require additional parameters
# False: Gauss-Seidl method; is to be set with the optional parameter omega (defult value is 1)
es.set_method(False, 1.1)

# Start the very calculation
result, iterations = es.compute() # compute the approximation of the equation using the iterative method

# Get information about the convergence of the iteration method
# Retuens True if the method converges, elsewise returns False
print("Method covergence: " + str(es.is_convergent()))

# Print the result of an approximation
pprint(result)

# Print the number of iterations
print("Iterations: " + str(iterations))

Method covergence: True
array([[0.9999992 , 0.99999869, 0.99999839, 0.99999825, 0.99999824,
        0.99999831, 0.99999843, 0.99999859, 0.99999877, 0.99999895,
        0.99999912, 0.99999928, 0.99999943, 0.99999956, 0.99999967,
        0.99999976, 0.99999984, 0.9999999 , 0.99999994, 0.99999998]])
Iterations: 38


## Conclusion



![plot](results.png "plot")

In conclusion, it appears that Gauss-Seidel method takes the smaller number of iterations to solve the system of equations comparely to Jacobi method. The decrease of _omega_ (< 1) value in the case of Gauss-Seidel method increases the number of iterations required. In contrast, the increse reduces (up to 2).